In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import multiprocessing
from snmfem.updates import dichotomy_simplex
import tqdm

In [ ]:
k = 5
p = 6400
span = np.logspace(-8,8,num=17)
iter = 100
TIMEOUT = 5

In [ ]:
for i in tqdm.tqdm(range(iter)) : 
    scale_num = np.random.choice(span,size=(k,p))
    num = scale_num * np.random.rand(k,p)
    scale_denum = np.random.choice(span,size=(k,p))
    denum = scale_denum * np.random.rand(k,p)
    dichotomy_simplex(num, denum)
    # process = multiprocessing.Process(target=dichotomy_simplex, name="fonction de ses morts", args=(num,denum))
    # process.start()
    # process.join(TIMEOUT)
    # if process.is_alive() :
    #     print('function terminated')
    #     N, D = num, denum
    #     process.terminate()
    #     process.join()
    #     break

In [ ]:
import hyperspy.api as hs 
import numpy as np
import snmfem.conf as conf
from pathlib import Path
import json

data_json = conf.SCRIPT_CONFIG_PATH / Path("dataset_EDXS_small.json")
with open(data_json,"r") as f :
    data_dict = json.load(f)

num_list = [0]
data_folder = conf.DATASETS_PATH / Path(data_dict["data_folder"])
samples = [data_folder / Path("sample_{}.npz".format(i)) for i in num_list ]

def load_data(sample) : 
    data = np.load(sample)
    X = data["X"]
    nx, ny, ns = X.shape
    Xflat = X.transpose([2,0,1]).reshape(ns, nx*ny)
    densities = data["densities"]
    phases = data["phases"]
    true_spectra_flat = np.expand_dims(densities, axis=1) * phases * data["N"]
    true_maps = data["weights"]
    k = true_maps.shape[2]
    true_maps_flat = true_maps.transpose([2,0,1]).reshape(k,nx*ny)
    G = data["G"]
    return Xflat, true_spectra_flat, true_maps_flat, G



In [ ]:
nx,ny = data_dict["weights_parameters"]["shape_2D"]
e_size = data_dict["model_parameters"]["e_size"]
X = load_data(samples[0])[0].T.reshape(nx,ny,e_size)
S = hs.signals.Signal1D(X)

In [ ]:
roi=hs.roi.RectangularROI(3,3,18,20)
S.plot(navigator="auto")
spim_ROI=roi.interactive(S)
    
sum_ROI=hs.interactive(spim_ROI.sum,
               event=spim_ROI.axes_manager.events.any_axis_changed,
               recompute_out_event=None)
sum_ROI.plot()